In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.1 MB/s eta 0:00:0

In [ ]:
!nvidia-smi

Thu Apr 27 22:31:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

### Load data

Link: https://huggingface.co/datasets/ccdv/cnn_dailymail

In [ ]:
dataset_cnn = load_dataset("ccdv/cnn_dailymail",'3.0.0')

split_lengths = [len(dataset_cnn[split])for split in dataset_cnn]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_cnn['train'].column_names}")


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Split lengths: [287113, 13368, 11490]
Features: ['article', 'highlights', 'id']


In [ ]:
print("\narticle:")

print(dataset_cnn["test"][0]["article"])

print("\nhighlights:")

print(dataset_cnn["test"][0]["highlights"])


article:
(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphrases su

In [ ]:
df_test = pd.DataFrame (dataset_cnn["test"]["article"], columns = ['article'])

In [ ]:
df_test['highlights'] = pd.DataFrame (dataset_cnn["test"]["highlights"])

In [ ]:
df_test

,article,highlights
0,"(CNN)James Best, best known for his portrayal ...","James Best, who played the sheriff on ""The Duk..."
1,(CNN)The attorney for a suburban New York card...,A lawyer for Dr. Anthony Moschetto says the ch...
2,(CNN)President Barack Obama took part in a rou...,"""No challenge poses more of a public threat th..."
3,Moscow (CNN)A Russian TV channel aired Hillary...,"Presidential hopeful's video, featuring gay co..."
4,(CNN)Marco Rubio is all in. The Republican se...,"Raul Reyes: In seeking Latino vote, Marco Rubi..."
...,...,...
11485,An axe-wielding robber has been jailed after ...,An axe-wielding robber has been jailed for att...
11486,Pope Francis has accepted the resignation of ...,Bishop Robert Finn failed to notify police abo...
11487,A breakthrough has been made in the developmen...,Bath and Yale University scientists reveal cle...
11488,John Carver says Newcastle’s hierarchy must tr...,John Carver believes Newcastle must sort out o...


In [ ]:
class cnn_test(torch.utils.data.Dataset):
    def __init__(self, df):
        self.data = df

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.data.iloc[idx].article	, self.data.iloc[idx].highlights

In [ ]:
test_data = cnn_test(df_test)

In [ ]:
for x, y in test_data:
  print(x)
  print(y)
  break

(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphrases such as "cuf

In [ ]:
import numpy as np

In [ ]:
from rouge_score import rouge_scorer 

In [ ]:
from tqdm import tqdm
results = {'rouge1': np.zeros(3), 'rouge2': np.zeros(3), 'rougeL': np.zeros(3)}

for data in tqdm(test_data,position=0, leave=True):
  article, ref = data

  inputs = tokenizer(article, return_tensors="pt", max_length=1024).to(device)
  summary_ids = model_pegasus.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=100)
  
  res = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL', 'rouge2'], use_stemmer=True)
  scores = scorer.score(res, ref)
  for key in scores.keys():
    results[key] += np.array(scores[key])



100%|██████████| 11490/11490 [5:16:59<00:00,  1.66s/it]


In [ ]:
len(test_data)

11490

In [ ]:
# average over all test data
for key in results.keys(): 
  results[key] /= len(test_data)
results

{'rouge1': array([0.46489289, 0.43186086, 0.43561237]),
 'rouge2': array([0.22031021, 0.2053699 , 0.20660281]),
 'rougeL': array([0.3278989 , 0.30392414, 0.30681007])}

In [ ]:
# Print Average F1 metrics 
print("average R1: ", results['rouge1'][2])
print("average R2: ", results['rouge2'][2])
print("average RL: ", results['rougeL'][2])

average R1:  0.43561236858089714
average R2:  0.20660280583974183
average RL:  0.30681007355593454
